In [0]:
# Recreate learner dataset
learner_data = [
    ("Alex", "Sales", "Module 1", "2025-06-01", 1, 70),
    ("Alex", "Sales", "Module 2", "2025-06-03", 1, 80),
    ("Bri", "Engineering", "Module 1", "2025-06-01", 1, 95),
    ("Bri", "Engineering", "Module 2", "2025-06-04", 1, 98),
    ("Casey", "CS", "Module 1", "2025-06-01", 1, 60),
    ("Casey", "CS", "Module 2", "2025-06-02", 0, None),
    ("Dev", "Sales", "Module 1", "2025-06-01", 0, None),
    ("Dev", "Sales", "Module 2", "2025-06-03", 0, None),
]

columns = ["learner", "team", "module", "date", "completed", "score"]

df = spark.createDataFrame(learner_data, columns)
df.show()


+-------+-----------+--------+----------+---------+-----+
|learner|       team|  module|      date|completed|score|
+-------+-----------+--------+----------+---------+-----+
|   Alex|      Sales|Module 1|2025-06-01|        1|   70|
|   Alex|      Sales|Module 2|2025-06-03|        1|   80|
|    Bri|Engineering|Module 1|2025-06-01|        1|   95|
|    Bri|Engineering|Module 2|2025-06-04|        1|   98|
|  Casey|         CS|Module 1|2025-06-01|        1|   60|
|  Casey|         CS|Module 2|2025-06-02|        0| null|
|    Dev|      Sales|Module 1|2025-06-01|        0| null|
|    Dev|      Sales|Module 2|2025-06-03|        0| null|
+-------+-----------+--------+----------+---------+-----+



In [0]:
from pyspark.sql.functions import avg, count, sum as spark_sum

# Group by learner and calculate progress metrics
progress_df = df.groupBy("learner").agg(
    spark_sum("completed").alias("modules_completed"),
    count("module").alias("total_modules"),
    avg("score").alias("avg_score")
)

# Calculate progress percentage
progress_df = progress_df.withColumn(
    "progress_percent", (progress_df["modules_completed"] / progress_df["total_modules"]) * 100
)

progress_df.show()


+-------+-----------------+-------------+---------+----------------+
|learner|modules_completed|total_modules|avg_score|progress_percent|
+-------+-----------------+-------------+---------+----------------+
|   Alex|                2|            2|     75.0|           100.0|
|    Bri|                2|            2|     96.5|           100.0|
|  Casey|                1|            2|     60.0|            50.0|
|    Dev|                0|            2|     null|             0.0|
+-------+-----------------+-------------+---------+----------------+



In [0]:
from pyspark.sql.functions import when

# Add a flag for at-risk learners
risk_df = progress_df.withColumn(
    "at_risk",
    when((progress_df["progress_percent"] < 50) | (progress_df["avg_score"] < 70), "YES").otherwise("NO")
)

risk_df.show()


+-------+-----------------+-------------+---------+----------------+-------+
|learner|modules_completed|total_modules|avg_score|progress_percent|at_risk|
+-------+-----------------+-------------+---------+----------------+-------+
|   Alex|                2|            2|     75.0|           100.0|     NO|
|    Bri|                2|            2|     96.5|           100.0|     NO|
|  Casey|                1|            2|     60.0|            50.0|    YES|
|    Dev|                0|            2|     null|             0.0|    YES|
+-------+-----------------+-------------+---------+----------------+-------+



In [0]:
# Save risk analysis as a Delta Table
risk_df.write.format("delta").mode("overwrite").saveAsTable("learner_progress_summary")


In [0]:
%sql
SELECT learner, progress_percent, avg_score, at_risk
FROM learner_progress_summary
ORDER BY progress_percent ASC;


learner,progress_percent,avg_score,at_risk
Dev,0.0,null,YES
Casey,50.0,60.0,YES
Alex,100.0,75.0,NO
Bri,100.0,96.5,NO


In [0]:
%sql
SELECT at_risk, COUNT(*) AS learner_count
FROM learner_progress_summary
GROUP BY at_risk;


at_risk,learner_count
YES,2
NO,2


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT learner, progress_percent
FROM learner_progress_summary;


learner,progress_percent
Alex,100.0
Bri,100.0
Casey,50.0
Dev,0.0


Databricks visualization. Run in Databricks to view.